In [1]:
## Ipython notebook to open emails and store them in a json file

In [1]:
import pandas as pd
import json
from os import listdir
import os
import re
import pymongo

In [4]:
filepath = './FlaskApp/data/maildir/'

In [141]:
import pymongo
## Set up Mongo DB
from pymongo import MongoClient

client = MongoClient()
db = client['email_subset']
collection = db['email']


In [265]:
collection.remove()

{u'n': 28290, u'ok': 1}

In [125]:
positions = pd.read_csv('FlaskApp/data/enron-mysqldump_v5.sql', header=None, quotechar="'", skipinitialspace=True)
positions.head(5)

email1 = positions[[3,8]]
email1.columns = ['email', 'position']
email2 = positions[[4,8]]
email2.columns = ['email', 'position']
email3 = positions[[5,8]]
email3.columns = ['email', 'position']
email4 = positions[[6,8]]
email4.columns = ['email', 'position']
full_email_list = pd.concat([email1, email2, email3, email4], axis=0)
full_email_list.reset_index()
pass

In [137]:
email = 'phillip.allen@enron.com'

a = full_email_list.position[full_email_list.email == email]
a.values[0]

'Manager'

In [258]:
## Main loop for processing
# for a given user, read in the emails, strip out various metrics, 


def readFiles(filepath, user, master):
    directories = listdir(filepath + user)
    
    ## Create list of emails
    walkthrough = os.walk(filepath+'/'+user)
    userEmails = []
    for i in walkthrough:
        for j in i[2]:
            email = i[0]+'/'+j

            email_id, parsedText = stripFiles(email)
            
            if not pd.isnull(parsedText['email_from_position']) and not pd.isnull(parsedText['email_to_position']):
                master = master.append({'FROM':parsedText['email_from_position'],
                               'TO':parsedText['email_to_position'],
                               'TEXT':parsedText['email_text']},
                              ignore_index=True)

    return master

def getPosition(email):
    return full_email_list.position[full_email_list.email == email].values[0]
    
    
def stripFiles(email):
    
    with open(email) as f:
        text = f.read()
    text = text.decode(errors='ignore').encode('utf-8', errors='ignore')
    
    email_id = re.sub('\.', '',
                      re.sub('\.Java.*', '', 
                             re.sub('Message-ID: <', '', 
                                    re.findall('Message-ID:.*\n',text)[0]), flags=re.DOTALL))
    
    email_from = re.findall('.*enron\.com', 
                            re.sub('From: ', '', 
                                   re.findall('From:.*\n',text)[0]))
    
    email_to = re.findall('.*enron\.com',
                          re.sub('\t','',
                                 re.sub('To: ', '',
                                        re.findall('To:.*:',text, flags=re.DOTALL)[0])))
    
    ccs = re.findall('Cc:.*\n',text)
    if ccs:
        email_ccs = re.findall('.*enron\.com', re.sub('Cc: ', '', ccs[0]))
    else:
        email_ccs = []
        
    email_subject = [re.sub('Subject: ', '',
                            re.findall('Subject:.*\n',text)[0])]

    email_text = re.sub('X-FileName:.*', '',
                         re.findall('X-FileName:.*',text, flags=re.DOTALL)[0])
    
    try:
        email_from_position = getPosition(email_from[0])
    except IndexError:
        email_from_position = None
    try:
        email_to_position = getPosition(email_to[0])
    except IndexError:
        email_to_position = None

    
    output = {'email_id':email_id,
              'email_from':email_from,
              'email_from_position':email_from_position,
              'email_to':email_to,
              'email_to_position':email_to_position,
              'email_text':email_text,
              'email_ccs':email_ccs,
              'email_subject':email_subject,
              #'full_text':text
             }
    
    return email_id, output



master = pd.DataFrame(columns=['FROM', 'TO', 'TEXT'])

users = listdir(filepath)
users = users

for user in users:
    master = readFiles(filepath, user, master)

    
with open('FlaskApp/data/subset.pkl', 'w+') as f:
    pickle.dump(master, f)

KeyboardInterrupt: 

In [268]:
%pwd

u'/Users/datascientist/Desktop/abergin/Email'

In [270]:
import pickle



In [264]:
master.FROM.unique()

array(['Manager', 'Director', 'Vice President', 'President', 'Employee',
       'Trader', 'Manager)', 'CEO', 'Managing Director', 'In House Lawyer'], dtype=object)

In [253]:
master[master.FROM.isnull()]

,FROM,TO,TEXT
122,NaN,Manager,\n\r\nPlease give me a call - 503-805-2117. I...
134,NaN,Employee,"\n\r\nIn regards to your E-mail, I have recent..."
188,NaN,Manager,\n\r\nPlease give me a call - 503-805-2117. I...
249,NaN,Manager,\n\r\nPlease give me a call - 503-805-2117. I...
630,NaN,Manager,\n\r\nThanks for making me work out yesterday ...
726,NaN,Manager,"\n\r\nJohn,\n\nAre you kidding! I'm lonely as..."
828,NaN,Manager,\n\r\nThanks for making me work out yesterday ...
843,NaN,Manager,\n\r\nThanks for making me work out yesterday ...
1125,NaN,Director,\n\r\n---------------------- Forwarded by Mary...
1133,NaN,Director,\n\r\n---------------------- Forwarded by Mary...


In [161]:
c = db.email.count()
c

193924

In [65]:
full_email_list.position.unique()

array([nan, 'Employee', 'CEO', 'Director', 'Trader', 'President',
       'Vice President', 'Manager', 'Managing Director', 'In House Lawyer',
       'Manager)'], dtype=object)